In [1]:
%load_ext autoreload
%autoreload 2
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

RANDOM_SEED = 42

housing = sklearn.datasets.fetch_california_housing()

x_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target, random_state=RANDOM_SEED)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(15480, 8) (15480,)
(5160, 8) (5160,)


# Baseline

In [2]:
regressor = DecisionTreeRegressor(
    random_state=RANDOM_SEED,
    min_samples_leaf=5,
    min_samples_split=20
)
regressor.fit(x_train, y_train)
baseline_acc = regressor.score(x_test, y_test)
print(baseline_acc)

0.6985753014500338


In [3]:
from decision_tree.DecisionTreeRegressor import CustomDecisionTreeRegressor

custom_regressor = CustomDecisionTreeRegressor()
custom_regressor.fit(x_train, y_train)
print(custom_regressor.score(x_test, y_test))



Node split: feature_index=0, threshold=5.03165, left_size=12147, right_size=3333
Node split: feature_index=0, threshold=3.0743, left_size=5883, right_size=6264
Node split: feature_index=2, threshold=4.200790513833992, left_size=2165, right_size=3718
Node split: feature_index=5, threshold=2.4993826802449393, left_size=676, right_size=1489
Node split: feature_index=0, threshold=2.2851, left_size=336, right_size=340
Node split: feature_index=2, threshold=3.3259634244836738, left_size=135, right_size=201
Node split: feature_index=4, threshold=2031.5, left_size=118, right_size=17
Node split: feature_index=6, threshold=37.905, left_size=99, right_size=19
Node split: feature_index=3, threshold=0.9566362557903062, left_size=10, right_size=89
Node split: feature_index=6, threshold=33.925, left_size=24, right_size=65
Node split: feature_index=3, threshold=1.1366673208707456, left_size=16, right_size=8
Node split: feature_index=4, threshold=1654.0, left_size=58, right_size=7
Node split: feature_i

In [4]:
from decision_tree.DecisionTreeRegressor import CustomDecisionTreeRegressor

custom_regressor_max_features = CustomDecisionTreeRegressor(max_features=2)
custom_regressor_max_features.fit(x_train, y_train)
print(custom_regressor_max_features.score(x_test, y_test))

Node split: feature_index=7, threshold=-121.86500000000001, left_size=3649, right_size=11831
Node split: feature_index=0, threshold=5.3957999999999995, left_size=2864, right_size=785
Node split: feature_index=6, threshold=37.925, left_size=1741, right_size=1123
Node split: feature_index=7, threshold=-122.305, left_size=558, right_size=1183
Node split: feature_index=5, threshold=2.612190928228664, left_size=301, right_size=257
Node split: feature_index=1, threshold=46.5, left_size=115, right_size=186
Node split: feature_index=1, threshold=19.5, left_size=16, right_size=99
Node split: feature_index=5, threshold=1.730505467203141, left_size=14, right_size=85
Node split: feature_index=0, threshold=3.4356, left_size=31, right_size=54
Node split: feature_index=5, threshold=2.0420288938178244, left_size=10, right_size=21
Node split: feature_index=4, threshold=1428.0, left_size=11, right_size=10
Node split: feature_index=6, threshold=37.605000000000004, left_size=18, right_size=36
Node split: 